In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import json
import re

In [23]:
sitemap = requests.get('https://www.bonappetit.com/sitemap.xml')

In [26]:
soup = BeautifulSoup(sitemap.content, "html.parser")

In [27]:
metapagelist = []
for i in soup.select("loc"):
    metapagelist.append(i.get_text())

In [28]:
len(metapagelist)

1035

In [29]:
pagelist = []
for page in metapagelist: 
    subpage = requests.get(page)
    soup = BeautifulSoup(subpage.content, 'html.parser')
    for i in soup.select("loc"):
        pagelist.append(i.get_text())

In [30]:
len(pagelist)

40167

In [33]:
pagelist[2]

'https://www.bonappetit.com/recipe/dry-rubbed-roast-turkey'

In [34]:
newpagelist = []
for page in pagelist: 
    if 'https://www.bonappetit.com/recipe' in page:
        newpagelist.append(page)

In [35]:
len(newpagelist)

17682

In [36]:
newpagelist[0]

'https://www.bonappetit.com/recipe/dry-rubbed-roast-turkey'

In [37]:
def getrecipe(recipeslist):
    recipes = []

    for page in recipeslist: 
        try: 
            req = requests.get(page)
            soup = BeautifulSoup(req.content, "html.parser")
            data = json.loads(soup.find('script', type='application/ld+json').text)
            name = data['name']
            ingredientlist = data['recipeIngredient']
            recipes.append([name, page, ingredientlist])
            print(f"Done with index # {recipeslist.index(page)}")
           
            # The try/except structure here has two benefits: 
            # - It guards against a patchy internet connection: It just keeps on rolling until it's back.
            # - It will automatically skip pages that do not conform to the same json format (recipeIngredient)
            
        except: 
            print("No success, moving on.")
        
        # Polite wait time
        wait_time = randint(1,4000)
        print("I will sleep for " + str(wait_time/4000) + " seconds.")
        sleep(wait_time/4000)
        
    return recipes

In [1]:
#bonap1 = getrecipe(newpagelist[0:5000])

In [2]:
#bonap2 = getrecipe(newpagelist[5000:10000])

In [43]:
ba1 = pd.DataFrame(bonap1)
ba2 = pd.DataFrame(bonap2)

In [44]:
bonapp_df = pd.concat([ba1, ba2], axis = 0)

In [45]:
bonapp_df.to_csv('bonapp1.csv', sep = '|', index = False)

In [3]:
#bonap3 = getrecipe(newpagelist[10000:15000])

In [4]:
#bonap4 = getrecipe(newpagelist[15000:])

In [ ]:
ba3 = pd.DataFrame(bonap3)
ba4 = pd.DataFrame(bonap4)
bonap = pd.concat([ba1, ba2, ba3, ba4], axis = 0)

In [ ]:
bonap.head()

In [ ]:
bonap.columns = ['name', 'url', 'ingredients']